In [ ]:
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/billingegroup/colab_notebooks_public/blob/main/crysthack_day1_python_packaging/packaging_tutorial.ipynb)

# space-group Mining: a hands on tutorial for training a deep neural net to yield the symmetry space-group from a given atomic pair distribution function (PDF)

Simon J. L. Billinge, Chia-Hao Liu, Yunzhe Tao, Hung Vuong, Ling Lan
*Columbia University, New York, NY*

## Introduction

This jupyter notebook contains a hands-on tutorial for training a deep neural net that, given an atomic pair distribution function (PDF) that encodes structural information it will suggest the symmetry space group of the 3D atomic structure that gave rise to the PDF signal.

The original work is described in (please cite that paper if you use this in your work):

> Chia-Hao Liu, Yunzhe Tao, Daniel J. Hsu, Qiang Du, and Billinge, Simon J. L. **Using a
machine learning approach to determine the space group of a structure from the atomic pair
distribution function**  *Acta Crystallogr. A* **75** (2019), pp. 633-643. 
doi: 10.1107/S2053273319005606. 
url: https://doi.org/10.1107/S2053273319005606

### The problem
In Materials science and chemistry we always want to know how the atoms are arranged in a material, the "structure" of the material.  For about 100 years we have been able to do that when the materials are crystals with periodic long-range order.  The first step is to do an x-ray diffraction experiment, indext it to find the unit cell shape and dimensions and the symmetry-space group from the *positions* of the Bragg peaks and any characteristically absent Bragg peaks.  We then obtain the positions of the atoms within the unit cell from the Bragg peak *intensities*.

If the structure of interest is not periodically long-range ordered, such as a nanoparticle, we can still measure the x-ray diffraction pattern but we can't take this crystallographic approach to solving the structure.  One path forward is to determine the atomic pair distribution function (PDF), which is the Fourier transform of the Bragg and diffuse scattering intensities and try and solve the structure from there.

The Fourier transform scrambles the information about the metric space (the unit cell and the space-group symmetry) and the decoration of that space (the positions of the atoms in the unit cell).  Before, they were separated into Bragg peak positions and intensities, respectively.  For example, there is no straightforward algorithmic way of getting the space-group information directly from the PDF, yet that information is still in the PDF since it is simply the FT of the diffraction data.

### The proposed solution

This is an idea Machine Learning problem.  It is a categorization problem (which space-group does this PDF come from), where we know that the information to do this categorization is in the data but we don't know how to make use of it to solve the problem.  Let's train a ML model to recognize the sg from a PDF.  Furthermore, there are databases of labelled data.  For example, structural databases exist that contain just about all known atomic structures, including the unit cell, the positions of the atoms, and the space-group.

We want a model that we can give as input a measured PDF and it gives as output the space-group.  We therefore choose for our feature vector the PDF itself, with the space-group of the underlying structure as the label.

## The approach
1. Collate a set of training data.
  1. From ICSD (or another structural database) pull all the structures and their space-groups
1. Munge the data
  1. Compute PDFs (using processing parameters that are somewhat experimentally realistic)
1. Carry out some Exploratory Data Analysis (EDA).  
  1. Plot the data so you know what it looks like
  1. Decide how long of a feature vector we want, what range of the PDF we want to give to the algorithm, what are
     reasonable parameters to use in the calculations
1. Munge some more
  1. find duplicate data or outliers
  1. normalize the data somewhat, for example, so that different space-groups are not over or under-represented
1. Choose a model.  It is common to start with simpler models such as, in this case, Linear Regression.  If that doesn't perform well enough explore a more fancy model, in this case a convolutional neural net.
1. Train the Model. ...
5. Evaluate the Model. ...
6. Parameter Tuning. ...
7. Make Predictions.

In the interests of time, we have already carried out (1.) and (2.).  The resulting PDFs are contained in the folder `pdf_training_data`


### 3. EDA

A. First we pick a one or two representatative datasets at random from the folder and plot them to see what they look like.

ModuleNotFoundError: No module named 'tensorflow'

### 3. EDA

B. Let's decide what we will use as the feature vector.  We know that we want it to be the "PDF" but what exactly?  Here are some questions we can think about.
1. What range of PDF?  
    * Longer means more information in the feature vector, shorter means quicker computation.   
    * Also, when people use the tool, what range of PDF will they have?  We don't want to build a tool noone can use
    * Shorter-range will more heavily weight local structure.  What if local structure is different than average structure?
1. What grid in the PDF?
    * The feature vectors must all be the same length (number of components).  In our case, it is the number of points in our PDF.
    * By changing the grid courseness we can get the same *r*-range of the PDF with a higher or lower dimensionality to our feature vector, so how do we choose.
    * We can use information theory and pick a grid that is close to the Nyquist-Shannon sampling grid.  This is the sparsest grid for the PDF that preserves all the information that was in the original diffraction pattern.  The grid-step is given by $2\pi/Q_{max}$.  Of course, we don't know $Q_{max}$ for some users' data, but we can use a value that (a) is the right one for our training data where we have control over this and (b) just oversamples what most users' data is probably going to look like.
1. We could ask more questions.  
    * For example, since we just want space-group and not lattice parameter, we could interpolate all data so that the first peak appears at 1 (i.e., normalize the vectors so that the nearest-neighbor bond length in all materials is unity.  We won't do that here, most materials have around the same atomic density, but we could.
    * ... anything else we can think of?

Summary, we will use an $r$-range of ???? and a feature vector dimension of 209.

Let's plot what our representative PDFs look like after this transformation...

In [ ]:
Please add code here

## 4. Munge some more: A duplicate datasets

Let's check the data for any duplicates.  

We are lucky here because we don't have to do it by hand.  We can just run a Pearson correlation between all the gr datasets in the directory to find any that are the same as each other... 

# Ling, if necessary can you create some duplicates on purpose and we can find them this way and remove them**

In [ ]:
Please add code here

Having found duplicate datasets we need to remove them...

In [ ]:
Please add code here

## 4. Munge some more: B data normalization

### Data normalization

_Hung: For the purpose of this tutorial, we have already processed PDF data as described in the previous sections. Let's load and explore the dataset._

In [ ]:
import os
import numpy as np

DATA_DIR = 'pdf_training_data'

X_data = np.load(os.path.join(DATA_DIR, 'X_data.npy'))
y_data = np.load(os.path.join(DATA_DIR, 'y_data.npy'), allow_pickle=True)

print('Total number of examples:', X_data.shape[0])

Here `X_data` is an array of dimension `(num_examples, 209)` with each row is the PDF of a structure and `y_data` is an vector of dimension `(num_examples, 1)`, where each entry is the space group of the structure.

In the original work we chose to categorize into the 45 most popular space-groups, which accounts for ~80% of all known structures in the inorganic database.  To make things easier and quicker here we will train our model to categorize structures from just 5 space groups, and when we test and use our model we will only give it PDFs from those space groups. The space groups in this tutorial are $P2_1/c, Pnma, Fm\bar{3}m, P\bar{1},Fd\bar{3}m$.

Let's enumerate how many structures we have in our dataset in each of these space groups:

In [ ]:
labels = list(np.unique(y_data))
for label in labels:
  print(f'The number of PDFs belong to space group {label}:', np.count_nonzero(y_data == label))

We could go with this distribution (this is how it was done in the original work, and then an assessment was done on the reliability of categerization by space-group to see if it correlated with the population size of each category) or we could sub-sample our data from each group to make sure each group is represented equally.

What is the best way to sub-sample?  We could do it randomly, but we also would like to know the variance encoded in each of our feature vectors within the group.  For example, if we have 10 PDFs labelled as coming from a particular space-group and we want to subsample this down to 5, but we find that 5 of the 10 PDFs are very similar to each other but the other 5 are very different, we will do better to take the PDFs with the same label that maximises the variance of the set.  We can use the Pearson correlation function again to determine this.  

Another related approach is to cluster the data with the same label, so similar PDFs are placed into blocks, and then use as a feature vector from each block that is the average of all the PDFs in that block. 

These are just some things to think about. At the end of the day, we want a model that predicts accurately, and unfortunately it is a bit of a trial and error game (basically try all these things and see which works best).

Here we will not do any data normalization and just take the full set.

## 5. Choose the model

Normally it is good to choose a linear model first that is quick to train to see if we are on the right track.  In this case we used Logistic Regression.  In the interests of time, in summary, we found that LR was ok, so we were on the right track, but not super accurate so we move to a deep neural net and picked a Convolutional Neural Net or CNN.

Unfortunately, there is not enough time to go into how CNN's are constructed and work in detail.  That is a course in its own right, not 5 mins in a hands-on lecture, so let's push on.

## 6. Space group mining using Convolutional Neural Network (CNN)

### Import the necessary libraries to build and train the model

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.layers import (Input, Convolution1D, Dense, MaxPooling1D,
                                    Flatten, Dropout, Activation, average,
                                    BatchNormalization)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

### Splitting the dataset into a training and a test dataset 

Typically for a machine learning model, we divide the dataset into a training, validation and a test set.

* _Training set_: We give the model the features and the label so it can learn its reduced set of internal parameters that give the best agreement of the label, given the feature vectors
* _Validation set_: We give the model features it hasn't seen and ask it to predict and check how well it is doing.  We then tweak things in the model so that it gives the best predictions of things it hasn't seen, rather than the best fit to what it has seen.  This is still a "training" activity but we are tweaking all the things we have control over and the model doesn't such as things like normalization that we discussed above, in the neural net the number of layers the structure of the layers and so on.  These are often called hyperparameters in ML.  To make the best use of our data we can use cross-validation where we hold out a small number, $m$ (usually 1-5) datasets in turn, but then do this $N/m$ times where $N$ is the total number of datasets.  This gives the algo the most data to learn on whilst still training to optimize prediction rather than fitting.
* _Testing set_: This is a representative set of data that we remove completely and we never let model see them during the training activity.  These provide an unbiased set of data that the model as *never* seen at any point during the training, and are used to validate the performance of hte fully tuned and trained model.


In this tutorial, we will divide the dataset into a training of $80\%$ of the dataset and a test/validation set of $20\%$ of the dataset. We employed a stratifying sampling strategy here to ensure that the training set and the test set have approximately the same distribution of data among the space groups, so that the model won't bias towards a particular space group with a lot of examples.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, stratify=y_data,
                                                    random_state=20, test_size=0.2)

In [ ]:
for label in labels:
  print(f'The number of training PDFs belong to space group {label}:', np.count_nonzero(y_train == label))
  print(f'The number of test PDFs belong to space group {label}:', np.count_nonzero(y_test == label))

Before we move on to building the model, we are going to convert the $y$ data into categorical format for the model:

In [ ]:
le = preprocessing.LabelEncoder()

le.fit(y_train)
y_train = le.transform(y_train)
y_train = to_categorical(y_train)

le.fit(y_test)
y_test = le.transform(y_test)
y_test = to_categorical(y_test)

In [ ]:
# reshaping the X data to the correct dimension for the model
X_train = X_train.reshape(-1, 209, 1)
X_test = X_test.reshape(-1, 209, 1)

### Create the model

The model contains two convolutional layers with batch normalization in each of them, followed by max pooling and drop-out with `drop_prob=0.5`. There is fully connected layers of 128 units with `relu` activation, followed by another prediction layer of `num_classes` units that outputs the probability of a PDF belonging to each of the 5 space groups. We choose `Adam` as our optimizer and `categorical_crossentropy` for our loss function.

In [ ]:
# declaring the models hyperparameters
input_shape = (209, 1)
filters1 = 256
filters2 = 64
kernel_size = 32
l2_lambda = 1e-5
ens_models = 1
dim = 209
num_classes = 5
hidden_dims = 128
drop_prob = 0.5

In [ ]:
def pdf_cnn(input_shape=(209, 1), 
            num_filters1=256,
            num_filters2=64,
            kernel_size=32,
            l2_lambda=1e-5,
            input_dims=209,
            hidden_dims=128,
            drop=0.5,
            num_classes=5):
    """
    Build a PDF space-group classifier with CNN

    The neural network architecture is from:

    https://doi.org/10.1107/S2053273319005606


    Parameters
    ----------
    input_shape: tuple
        dimension of input PDF signal, default to (209, 1) which corresponds
        to r-range of 1.5 to 30 A
    num_filter1: int
        number of filters in 1st convolution layer, default to 256
    num_filter2: int
        number of filters in 2nd convolution layer, default to 64
    kernel_size: int
        size of the convolution filter, default to 32
    ens_model: int
        number of ensemble models to train, default to 3
    l2_lambda: float
        strength of l2-regularization for kernel, default to 1e-5
    input_dims: int
        dimension of input data, default to 209
    hidden_dims: int
        dimension of the hidden layer followed by convolution layers,
        default to 128
    drop: float
        dropout ratio, default to 0.5
    num_classes: int
        total number of space-group labels, default to 5

    Returns
    -------
    model: tensorflow.keras.models.Sequential
        trainable CNN model for PDF space-group classifier

    """
    inputs = Input(shape=input_shape)
    inp_bn = BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(inputs)
    conv1 = Convolution1D(num_filters1, kernel_size, padding='same', activation='relu',
                          kernel_initializer='he_uniform',
                          kernel_regularizer=regularizers.l2(l2_lambda))(inp_bn)
    bn1 = BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(conv1)
    conv2 = Convolution1D(num_filters2, kernel_size, padding='same',
                          activation='relu', kernel_initializer='he_uniform',
                          kernel_regularizer=regularizers.l2(l2_lambda))(bn1)
    bn2 = BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(conv2)
    pool = MaxPooling1D()(bn2)
    drop1 = Dropout(drop)(pool)
    flat = Flatten()(drop1)
    dense = Dense(hidden_dims, activation='relu', kernel_regularizer=regularizers.l2(l2_lambda),
                  kernel_initializer='he_uniform')(flat)
    bn3 = BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(dense)
    drop2 = Dropout(drop)(bn3)
    out = Dense(num_classes, activation='softmax',
                kernel_regularizer=regularizers.l2(l2_lambda),
                kernel_initializer='glorot_uniform')(drop2)

    model = Model(inputs=inputs, outputs=out)
    # compiling the model
    model.compile(loss='categorical_crossentropy',
                optimizer=Adam(lr=5e-4),
                metrics=['accuracy'])
    return model

In [ ]:
model = pdf_cnn(input_shape=input_shape,
                num_filters1=filters1,
                num_filters2=filters2,
                kernel_size=kernel_size,
                l2_lambda=l2_lambda,
                input_dims=dim,
                hidden_dims=hidden_dims,
                drop=drop_prob,
                num_classes=num_classes)

model.summary()

### Train the model

In [ ]:
epochs = 50
batch_size = 32

history = model.fit(X_test, y_test, epochs=epochs, batch_size=batch_size, 
                    validation_data=(X_test, y_test), verbose=1)

### Visualize the results

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

_Hung: Is there anything else we need to add here? Maybe we can talk about how in Tim's paper we trained on a lot more data and more space groups, using learning rate decay, ensemble averaging? Also a confusion matrix plot here might be a good idea?_

## Hung and Ling, can you take it from here.  I think the story should pretty much follow Tim's code from here.  Also, if possible, fill in all the ???'s and correct any incorrectstatements I have made above.